## Лабораторная работа № 2 
## Машинное обучение на больших данных с использованием фреймворка Apache Spark и библиотеки SparkML

### Часть 1

В данной части работы рассмотрены:
* подготовка признаков для рашения задачи **линейной регрессии**;
* создание и обучение модели линейной регрессии;
* оценка качества модели.

#### Запуск `Spark`-сессии

Подключаем необходимые библиотеки.

In [35]:
import os
import csv
from pyspark.sql import SparkSession, DataFrame
from pyspark import SparkConf
from pyspark.ml.feature import VectorAssembler, StringIndexer, OneHotEncoder, MinMaxScaler, Binarizer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, CrossValidatorModel, ParamGridBuilder
from pyspark.ml import Pipeline
from pyspark.sql.functions import col
from pyspark.sql.types import DoubleType
import seaborn as sns
import matplotlib.pyplot as plt

Сформируем объект конфигурации для `Apache Spark`, указав необходимые параметры.

In [36]:
def create_spark_configuration() -> SparkConf:
    """
    Создает и конфигурирует экземпляр SparkConf для приложения Spark.

    Returns:
        SparkConf: Настроенный экземпляр SparkConf.
    """
    # Получаем имя пользователя
    user_name = os.getenv("USER")
    
    conf = SparkConf()
    conf.setAppName("lab 2 Test")
    conf.setMaster("yarn")
    conf.set("spark.submit.deployMode", "client")
    conf.set("spark.executor.memory", "12g")
    conf.set("spark.executor.cores", "8")
    conf.set("spark.executor.instances", "2")
    conf.set("spark.driver.memory", "4g")
    conf.set("spark.driver.cores", "2")
    conf.set("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.0")
    conf.set("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    conf.set("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkCatalog")
    conf.set("spark.sql.catalog.spark_catalog.type", "hadoop")
    conf.set("spark.sql.catalog.spark_catalog.warehouse", f"hdfs:///user/{user_name}/warehouse")
    conf.set("spark.sql.catalog.spark_catalog.io-impl", "org.apache.iceberg.hadoop.HadoopFileIO")

    return conf

Создаём сам объект конфигурации.

In [37]:
conf = create_spark_configuration()

Создаём и выводим на экран сессию `Apache Spark`. В процессе создания сессии происходит подключение к кластеру `Apache Hadoop`, что может занять некоторое время.

In [ ]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

#### Загрузка датасета

Укажем базу данных, которая была создана в первой лабораторной работе.

In [39]:
database_name = "ivanov_database"

Установим созданную базу данных как текущую.

In [40]:
spark.catalog.setCurrentDatabase(database_name)

Прочитаем таблицу с **предобработанным датасетом** и загрузим её в `Spark Dataframe`.

In [41]:
df = spark.table("sobd_lab1_processed_table")

Выведем прочитанную таблицу на экран.

In [ ]:
df.show()

Вспомним описание столбцов и параметры датасета, проанализированные в первой лабораторной работе.

| Название столбца          | Расшифровка |  Тип признака и его характеристики
| ------------------------- | ------------- | ------------------ |
| vin                       | Идентификационный номер автомобиля | категориальный, уникальный |
| body_type                 | Тип кузова автомобиля (кабриолет, хэтчбек, седан и т.д.)  | категориальный, 10 категорий (одна неизвестная: NULL) |
| daysonmarket              | Количество дней, прошедших с момента первого размещения автомобиля на сайте | количественный, непрерывный, интервал значений [0, 500] |
| horsepower                | Мощность двигателя в лошадиных силах | количественный, непрерывный, интервал значений [0, 400] |
| maximum_seating           | Максимальное количество посадочных мест | количественный, дискретный, интервал значений [2, 15] |
| mileage                   | Величина пробега автомобиля | количественный, непрерывный, интервал значений [0, 200000], большинство значений < 100 |
| price                     | Цена автомобиля | количественный, непрерывный, интервал значений [0, 60000] |
| wheel_system              | Тип привода | категориальный, 5 категорий |
| is_any_cert               | Является ли автомобиль сертифицированным (любым способом) | бинарный, true << false |
| contains_Alloy Wheels     | Имеются ли в автомобиле легкосплавные диски | бинарный |
| contains_Backup Camera    | Имеется ли в автомобиле резервная камера | бинарный |
| contains_Bluetooth        | Имеется ли в автомобиле поддержка Bluetooth | бинарный |
| contains_Heated Seats     | Имеется ли в автомобиле подогрев сидений | бинарный |
| contains_Sunroof/Moonroof | Имеется ли в автомобиле люк | бинарный |
| age                       | Количество лет, прошедших с года выпуска автомобиля до 2024 года | количественный, непрерывный, интервал значений [3, 43] |

Вспомним схему данных.

In [ ]:
df.printSchema()

Вычислим количество строк в датафрейме.

In [ ]:
df.count()

#### Постановка задачи линейной регрессии

Для датасета, заданного представленными колонками, требуется построить модель линейной регрессии для оценки **стоимости автомобиля** по всем остальным признакам. 

Для оценки качества обучения следует использовать метрики $RMSE$ и $R^2$.

#### Подготовка и кодирование признаков

Для корректной работы трансформеров преобразуем столбец `mileage` к типу `DoubleType`.

In [45]:
df = df.withColumn("mileage", col("mileage").cast(DoubleType()))

Отделим от датасета некоторую часть объёмом примерно 1000 строк, и сохраним её на диске как локальный `csv`-файл. Он понадобится в следующей лабораторной работе.

In [46]:
def save_sample_to_csv(data: DataFrame, file_path: str, 
                       sample_size: int = 1000) -> DataFrame:
    """
    Сохраняет первые `sample_size` строк из DataFrame в CSV-файл 
    на драйвере и возвращает DataFrame с оставшимися данными.

    Args:
        data (DataFrame): DataFrame, из которого нужно извлечь
            строки.
        file_path (str): Путь для сохранения CSV-файла.
        sample_size (int): Количество строк для сохранения
            (по умолчанию 1000).

    Returns:
        DataFrame: DataFrame с оставшимися данными.
    """
    # Определяем пропорции для разделения
    sample_fraction = sample_size / data.count()
    remaining_fraction = 1 - sample_fraction

    # Разделяем DataFrame на два непересекающихся набора данных
    sample_data, remaining_data = data.randomSplit(
        [sample_fraction, remaining_fraction]
    )

    # Сохраняем извлеченные строки в CSV-файл на драйвере
    try:
        with open(file_path, mode="w", newline="") as file:
            writer = csv.writer(file)

            # Записываем заголовок
            writer.writerow(data.columns)

            # Записываем строки
            for row in sample_data.take(sample_size):
                writer.writerow(row)
        print(f"Файл \"{file_path}\" с данными успешно создан.")

    except Exception as e:
        print(f"Ошибка при записи файла: {e}")

    return remaining_data

Определяем путь для сохранения `csv`-файла.

In [ ]:
path = "streaming-data.csv"

df = save_sample_to_csv(data=df, file_path=path, sample_size=1000)

Оцениваем, сколько строк в датасете осталось.

In [ ]:
df.count()

Разделим датасет на обучающую и тестовую выборки.

In [49]:
train_df, test_df = df.randomSplit([0.8, 0.2])

In [ ]:
print(f"Train dataset size: {train_df.count()}")
print(f"Test  dataset size: {test_df.count()}")

Понятно, что **идентификационный номер** автомобиля не оказывает влияния на цену. Использовать его в модели нет смысла.

Остальные признаки сгруппируем по их типу:

* **Категориальные** признаки не содержат большого количества категорий, закодируем их `one-hot`-кодировкой.
* **Бинарные** признаки представлены значениями `true` / `false`, которые могут быть интерпретированы как единица и нуль. Поэтому, в кодировании не нуждаются.
* **Количественные** признаки нужно нормализовать / стандартизировать, перед тем, как передавать их в модель.
* Среди количественных признаков выделяется `mileage`, который по своим значениям больше напоминает бинарный. **Бинаризуем** его по порогу `100`.

In [51]:
categorical_features = ["body_type", "wheel_system"]
binary_features = [
    "is_any_cert", "contains_Alloy Wheels", "contains_Backup Camera",
    "contains_Bluetooth", "contains_Heated Seats", "contains_Sunroof/Moonroof"
]
numeric_features = ["daysonmarket", "horsepower", "maximum_seating", "age"]
binarizable_feature = "mileage"

Создадим конвейер обработки данных, включающий модель линейной регрессии.

In [52]:
def create_pipeline(categorical_features: list[str], numeric_features: list[str], 
                    binary_features: list[str], binarized_col: str, 
                    threshold: float, label_col: str, max_iter: int) -> Pipeline:
    
    # Формируем названия колонок для преобразованных признаков
    indexed_categorical_features = [f"{feature}_index" for feature in categorical_features]
    onehot_categorical_features = [f"{feature}_ohe" for feature in categorical_features]
    
    # Выполняем преобразования данных
    string_indexer = StringIndexer(inputCols=categorical_features,
                                   outputCols=indexed_categorical_features,
                                   handleInvalid="keep")
    onehot_encoder = OneHotEncoder(inputCols=indexed_categorical_features,
                                   outputCols=onehot_categorical_features,
                                   dropLast=True,
                                   handleInvalid="keep")
    vector_num_assembler = VectorAssembler(inputCols=numeric_features,
                                           outputCol="numeric_vector")
    numeric_scaler = MinMaxScaler(inputCol="numeric_vector",
                                   outputCol="numeric_vector_scaled")
    binarizer = Binarizer(inputCol=binarized_col,
                          outputCol="mileage_binary",
                          threshold=threshold)
    vector_all_assembler = VectorAssembler(
        inputCols=(onehot_categorical_features + binary_features +
                   ["numeric_vector_scaled", "mileage_binary"]),
        outputCol="features"
    )

    # Создаем модель линейной регрессии
    linear_regression = LinearRegression(featuresCol="features",
                                         labelCol=label_col,
                                         predictionCol="prediction",
                                         standardization=False,
                                         maxIter=max_iter)

    # Создаем конвейер
    pipeline = Pipeline(stages=[
        string_indexer, onehot_encoder, vector_num_assembler,
        numeric_scaler, binarizer, vector_all_assembler,
        linear_regression
    ])

    return pipeline

In [53]:
pipeline = create_pipeline(categorical_features=categorical_features,
                           numeric_features=numeric_features,
                           binary_features=binary_features,
                           binarized_col="mileage",
                           threshold=100,
                           label_col="price",
                           max_iter=15)

#### Обучение модели

Выполним **подбор гиперпараметров** модели линейной регрессии с помощью кросс-валидации на сетке.

Создаем сетку параметров для кросс-валидации, получив объект `LinearRegression` из конвейера.

In [54]:
param_grid = ParamGridBuilder() \
    .addGrid(pipeline.getStages()[-1].regParam, [0.01, 0.1, 1.0]) \
    .addGrid(pipeline.getStages()[-1].elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

Создаем экземпляр `RegressionEvaluator` для оценки модели.

In [55]:
cv_evaluator = RegressionEvaluator(labelCol="price",
                                   predictionCol="prediction",
                                   metricName="rmse")

Создаем объект `CrossValidator`.

In [56]:
cross_validator = CrossValidator(estimator=pipeline,
                                 estimatorParamMaps=param_grid,
                                 evaluator=cv_evaluator,
                                 numFolds=5)

Обучаем модель конвейера с использованием кросс-валидации.

In [ ]:
cv_model = cross_validator.fit(train_df)

Выведем параметры **лучшей** модели, определенной в ходе кросс-валидации.

In [58]:
def get_best_model_params(cv_model: CrossValidatorModel) -> dict[str, float]:
    """
    Получает параметры лучшей модели из объекта CrossValidatorModel.

    Args:
        cv_model (CrossValidatorModel): Объект CrossValidatorModel, 
            содержащий лучшую модель.

    Returns:
        Dict[str, float]: Параметры лучшей модели.
    """
    best_model = cv_model.bestModel
    best_params = {
        "regParam": best_model.stages[-1].getRegParam(),
        "elasticNetParam": best_model.stages[-1].getElasticNetParam(),
        "maxIter": best_model.stages[-1].getMaxIter()
    }
    return best_params

In [ ]:
for key, value in get_best_model_params(cv_model=cv_model).items():
    print(f"{key}: {value}")

#### Анализ процесса обучения

Визуализируем изменение ошибки модели в ходе обучения и рассчитаем метрики на обучающем датасете.

In [60]:
def plot_training_summary(cv_model: DataFrame) -> None:
    """
    Отображает графики зависимости значения ошибки от номера итерации на
    обучающей выборке, а также значения метрик RMSE и R^2.

    Args:
        cv_model (DataFrame): Обученная модель с использованием кросс-валидации.
    """
    # Получаем лучшую модель
    best_model = cv_model.bestModel

    # Получаем информацию о процессе обучения
    training_summary = best_model.stages[-1].summary

    # Получаем значения ошибки для каждой итерации
    objective_history = training_summary.objectiveHistory

    # Строим график зависимости значения ошибки от номера итерации
    plt.figure(figsize=(10, 6))
    sns.lineplot(x=range(len(objective_history)), 
                 y=objective_history, 
                 marker='o')
    plt.xlabel('Итерация')
    plt.ylabel('Ошибка')
    plt.title("Зависимость значения функции ошибки от номера итерации")

    # Получаем значения метрик
    rmse = training_summary.rootMeanSquaredError
    r2 = training_summary.r2

    # Добавляем значения метрик на график
    plt.text(0.95, 0.95, f"RMSE: {rmse:.2f}\nR^2: {r2:.2f}",
             transform=plt.gca().transAxes, ha='right', va='top',
             bbox=dict(facecolor='white', alpha=0.8), zorder=5)
    plt.grid()

    plt.show()

In [ ]:
plot_training_summary(cv_model)

#### Проверка обобщающей способности модели

Выполним предсказания на тестовой выборке. 

Перегруппируем колонки датафрейма, переставив столбец с ценой автомобиля в конец, чтобы его значения было удобно сравнивать с предсказанными.

In [ ]:
# Получаем датасет предсказаний
test_df_predictions = cv_model.transform(test_df)

# Извлекаем список колонок, устанавливаем цену на последнее место
right_columns_order = test_df_predictions.columns
right_columns_order.remove("price")
right_columns_order.append("price")

# Изменяем последовательность колонок и выводим датафрейм
test_df_predictions = test_df_predictions.select(*right_columns_order)
test_df_predictions.show()

Создадим функцию оценки модели: расчета метрик для некоторого датасета, как правило, тестового.

In [63]:
def evaluate_model(data: DataFrame, metric_name: str) -> float:
    """
    Оценивает модель с использованием указанной метрики.

    Args:
        data (DataFrame): DataFrame, содержащий предсказания и фактические метки.
        metric_name (str): Название метрики для оценки (например, "rmse", "r2").

    Returns:
        float: Значение указанной метрики.
    """
    evaluator = RegressionEvaluator(labelCol="price", 
                                    predictionCol="prediction", 
                                    metricName=metric_name)
    metric_value = evaluator.evaluate(data)
    return metric_value

Оценим модель на тестовой выборке.

In [ ]:
test_rmse = evaluate_model(test_df_predictions, "rmse")
test_r2 = evaluate_model(test_df_predictions, "r2")

print(f"RMSE on test data: {test_rmse:.2f}")
print(f"R^2 on test data: {test_r2:.2f}")

Метрики весьма неплохие для линейной модели!

#### Сохранение модели

Зададим директорию студента в `HDFS`, в которой будет сохранена обученная модель.

In [65]:
student_hdfs_folder = "ivanov_directory"

In [ ]:
# Получаем имя пользователя
user_name = os.getenv("USER")

# Путь модели в HDFS
model_hdfs_path = f"hdfs:///user/{user_name}/{student_hdfs_folder}/models/lr-model"

# Сохраняем модель конвейера в HDFS
try:
    cv_model.bestModel.save(model_hdfs_path)
    print(f"Модель успешно сохранена в \"{model_hdfs_path}\"")
except Exception as e:
    print(f"Ошибка при сохранении модели: {e}")

Не забываем завершать `Spark`-сессию.

In [66]:
spark.stop()